In [1]:
#EJECUTAR UNA SOLA VEZ, EL PROGRAMA LE PEDIRÁ REINICIAR SESIÓN, HÁGALO Y DESPUES EJECUTE LA SIGUIENTE LÍNEA
# 🔥 Desinstalar versiones conflictivas
!pip uninstall -y numpy pmdarima scipy statsmodels

# ✅ Instalar versiones compatibles
!pip install numpy==1.24.3
!pip install scipy==1.10.1
!pip install statsmodels==0.14.0
!pip install --no-cache-dir pmdarima==2.0.3


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: scipy 1.16.0
Uninstalling scipy-1.16.0:
  Successfully uninstalled scipy-1.16.0
Found existing installation: statsmodels 0.14.5
Uninstalling statsmodels-0.14.5:
  Successfully uninstalled statsmodels-0.14.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 15.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires scipy>=1.11.1, which is not installed.
clarabel 0.11.1 requires scipy, which is not installed.
arviz 0.22.0 requires scipy>=1.11.0, which is not installed.
matplotlib-venn 1.1.2 requires scipy, which is not installed.
jax 0.5.2 requires scipy>=1.11.1, which is not installed.
lightgbm 4.6.0 requires scipy, which is not installed.
scikit-image 0.25.2 requires scipy>=1.11.4, which i

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 45.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
plotnine 0.14.5 requires statsmodels>=0.14.0, which is not installed.
tsfresh 0.21.0 requires statsmodels>=0.13, which is not installed.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
arviz 0.22.0 requires numpy>=1.26.0, but you have numpy 1.24.3 which is incompatible.
arviz 0.22.0 requires scipy>=1.11.0, but you have scipy 1.10.1 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
scikit-image 0.25.2 req

In [ ]:
#EJECÚTELA SOLO SI EN LA LÍNEA ANTERIOR
# This ensures we are not introducing complexities with specific versions yet.
print("Attempting simple installation of pmdarima...")
!pip install pmdarima --quiet

# Now, let's re-check the Python path and installed packages in this
# potentially fresh environment.
import sys
print("\nPython executable after simple install:", sys.executable)
print("Python version after simple install:", sys.version)
print("Python path after simple install:", sys.path)

print("\nVerifying installation with simple install:")
!pip list | grep pmdarima


# Attempt the import again.
try:
    from pmdarima.arima import auto_arima
    print("\n✅ ¡pmdarima está listo para usarse!")
except ModuleNotFoundError as e:
    print(f"\n❌ ModuleNotFoundError: {e}")
    print("pmdarima module was not found in the Python path even after a simple install and factory reset.")
    print("This indicates a persistent environment issue on the platform or with the package itself.")
except Exception as e:
    print(f"\n❌ An unexpected error occurred during import: {e}")

Attempting simple installation of pmdarima...

Python executable after simple install: /usr/bin/python3
Python version after simple install: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Python path after simple install: ['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython']

Verifying installation with simple install:
pmdarima                              2.0.3


# 🗓️ **Semana 4: Análisis de Series de Tiempo con Modelos ARIMA, ARMA y SARIMAX** 📈
---
## 🎯 **Objetivo de la Semana**
Durante esta semana, exploraremos el **análisis de series de tiempo**, una herramienta esencial en la **analítica predictiva** que nos permite comprender, modelar y predecir comportamientos basados en datos recogidos en intervalos de tiempo regulares.

---

## 📊 **Contenidos**
Nos enfocaremos en tres técnicas fundamentales para modelar y predecir **series temporales**:

- **ARIMA (Autoregressive Integrated Moving Average)**: Un modelo potente que combina **autoregresión (AR)**, **promedio móvil (MA)** y un componente de **integración** para hacer que las series se vuelvan estacionarias. Este modelo es ideal para datos que presentan **tendencias**.
  
- **ARMA (Autoregressive Moving Average)**: Similar a ARIMA pero sin el componente de integración. ARMA es utilizado para series temporales que ya son **estacionarias**, combinando términos **autoregresivos** y de **promedio móvil** para capturar la dinámica temporal.

- **SARIMAX (Seasonal Autoregressive Integrated Moving Average with Exogenous Variables)**: Este modelo extiende ARIMA para capturar la **estacionalidad** y permite incorporar **variables exógenas** que afectan el comportamiento de la serie. Es útil cuando el objetivo es modelar patrones estacionales y entender cómo factores externos influyen en la serie.

---
## 🤔 **¿Por Qué Aprender Sobre Series de Tiempo?**
Las **series de tiempo** son cruciales para el análisis predictivo porque permiten prever el comportamiento futuro basándose en patrones históricos. Este tipo de análisis es útil en **finanzas** (por ejemplo, para predecir precios de acciones), **ventas** (para estimar la demanda de productos) o incluso en la **meteorología** (para predecir el clima).

Durante esta semana, aprenderemos a:
- Entender los conceptos básicos de las **series de tiempo**.
- Aplicar modelos como **ARIMA, ARMA y SARIMAX** para modelar y predecir datos.
- Evaluar la **calidad del ajuste** y realizar **predicciones** que puedan servir para la **toma de decisiones** empresariales.

---

## 🚀 **Actividad Práctica**
Trabajaremos con un conjunto de datos que contiene información histórica sobre valor de las acciones de una empresa. Utilizaremos nuestros modelos para:
1. **Identificar patrones y estacionalidades**.
2. **Ajustar el mejor modelo** para los datos.
3. **Hacer predicciones** para el próximo año y así apoyar las decisiones de negocio.

Esta semana nos adentraremos en el mundo de las **predicciones temporales**, una habilidad que nos permitirá anticipar cambios y preparar estrategias más efectivas a nivel empresarial.


---
---
# 🤓 **Empecemos un poco de teoría**

# **Las Series de tiempo**

Una de las principales preguntas que puede surgir al momento de iniciar esta semana, es el por qué de esta nueva sección que analiza de forma diferente los datos cuando ya hemos cubierto problemas donde la variable es de corte numérica y contínua en el modelo de Regresión Lineal.

La principal respuesta es que el análisis de series de tiempo reconoce la autocorrelación entre eventos de tiempo, que  pudiese existir, mientras que en el modelo de Regresión Lineal asumimos, o minimizamos, la independencia de los errores de serie analizada. Por ello, una de las principales diferencias entre el enfoque de regresión lineal y el de series de tiempo es que en el primero, estudiamos a una variable que está ordenada en el tiempo, mientras que en el modelo de Regresión Lineal, no siempre es así. Recordemos el ejemplo del precio de los inmuebles, que consistía en el análisis de esos datos sin la variable de tiempo.

Sin embargo, esto no quiere decir que _no_ podemos aplicar modelos de regresión a series temporales, si quisiéramos tomar esa vía, lo que se recomienda es que la autocorrelación sea nula o baja, o se encuentre una variable oculta que explique la autocorrelación. De igual forma, se recomienda incluir variables transformadas, como la diferenciación de la serie, o variables estacionales ficticias para controlar por los componentes de tendencia o cíclicos en el modelo de regresión, pero esto lo acerca más al análisis de series de tiempo que al de regresión.


De hecho, una de las razones para usar modelos series de tiempo, es que es poco probable que se cumplan los supuestos de error que subyacen en el modelo de regresión lineal, en particular la falta de correlación serial.



**Conceptos importantes**

- Regresión: Es una herramienta utilizada para evaluar la relación de una variable dependiente en relación con múltiples variables independientes, es decir, su objetivo es inferir las relaciones que existen entre la variable dependiente y las variables independentes, por lo que la regresión analizará la media de la variable dependiente en relación con los cambios en las variables independientes
- Serie de tiempo: Mide los datos de una serie durante un período de tiempo específico, por lo que el valor del período de tiempo anterior es (casi siempre) un buen predictor del valor del período actual


**¿Entonces qué es una serie de tiempo y dónde está su valor agregado?**

Series de tiempo es el estudio de aquellos datos que *dependen del tiempo o que cambian con el tiempo*. Este particular nos permite usar como insumo a los valores pasados para predecir el futuro a través de patrones que se repiten con el tiempo. Cabe recalcar que esta predicción, también incluirá al tiempo como variable.

Una forma sencilla de evaluar si una serie podría ser un buen candidato para el análisis por series de tiempo consiste en evaluar si la serie se recopila en intervalos de tiempo regulares, si adicionalmente al graficarla, uno de los ejes corresponde al tiempo, estaríamos frente a un buen candidato para este tipo de análisis.

Por lo general, en los modelos regulares iniciamos la modelización ignorando la causa de la variación de nuestra variable dependiente, mientras que en las series temporales, algunas de las causas de variación del modelo son deterministas.

Un dato importante de las series de tiempo es que requieren que los datos sean estacionarios, en otras palabras, que la media y la desviación estándar de la serie sean constantes a lo largo del tiempo.

---
---

# 🏡 **Caso de Estudio: Predicción del precio de cierre de las acciones de Tesla**

## **Contexto del problema:**

Supongamos que somos **analistas financieros** para una empresa de inversión que busca ayudar a **inversores** a comprender mejor el comportamiento de las acciones de **Tesla**. Nuestro objetivo es construir un **modelo predictivo robusto** que permita a nuestros clientes tomar **decisiones informadas de compra y venta** de acciones. Utilizaremos técnicas avanzadas de **análisis de series de tiempo** para predecir el **precio de cierre** de las acciones de Tesla, considerando la volatilidad del mercado y la popularidad de la empresa.

Queremos que este análisis sea una herramienta que **inversores particulares y gestores de fondos** puedan utilizar para optimizar sus estrategias y anticipar el comportamiento de los precios de las acciones de Tesla, una de las compañías tecnológicas más dinámicas e influyentes del mercado actual.


### 📊 **Sobre la Base de Datos**

| Columna                | Descripción                                                                                 |
|------------------------|---------------------------------------------------------------------------------------------|
| `precio_apertura`      | Precio de apertura de la acción de Tesla en el día.                                         |
| `precio_max`           | Precio máximo alcanzado durante el día de transacción.                                      |
| `precio_min`           | Precio mínimo alcanzado durante el día de transacción.                                      |
| `precio_cierre`        | Precio de cierre de la acción al final del día.                                             |
| `precio_ajustado_cierre` | Precio de cierre ajustado para dividendos y splits.                                       |
| `Volumen`              | Número de acciones negociadas durante el día.                                               |
| `fecha`                | Fecha correspondiente al día de transacción.                                                |

**Para lo cual seguiremos los siguientes pasos:**

### 📝 1. Definición del Problema

- **Objetivo**: Predecir el **precio de cierre** de las acciones de Tesla, proporcionando a los inversores información relevante para maximizar sus ganancias y reducir el riesgo.
- **Beneficio**: Brindar insights a inversores para que puedan anticiparse a las tendencias del mercado y tomar decisiones de inversión informadas, aprovechando la volatilidad del precio de Tesla para obtener mejores rendimientos.


### 🔍 2. Análisis Exploratorio de Datos (EDA)

- **Descripción de los Datos**:
  - Analizar las estadísticas descriptivas para identificar patrones en el comportamiento de los precios (`precio_apertura`, `precio_max`, `precio_min`, `precio_cierre`) y el **volumen de acciones negociadas**.
  - Determinar la **variabilidad** de los precios para comprender la volatilidad y las posibles oportunidades de inversión.

- **Visualización de la Distribución**:
  - Utilizar **la descomposición de la serie** para evaluar cada una de ellas.


### 🛠️ 3. Preparación de los Datos

- **Transformación de Fechas**: Convertir la columna `fecha` a un formato adecuado para el análisis de series de tiempo, extrayendo **mes**, **año**, y **día de la semana** para identificar patrones temporales.

- **Filtrado de datos**: seleccionar la base de datos idónea para predecir valores futuros


### 📈 4. Análisis de Series de Tiempo

- **Descomposición de la Serie Temporal**:
  - Descomponer la serie temporal de precios en **tendencia**, **estacionalidad** y **ruido**, utilizando enfoques **aditivos** y **multiplicativos**. Esto ayudará a entender los componentes que influyen en el precio y su comportamiento a lo largo del tiempo.

- **Modelos Predictivos a Implementar**:
  - **Modelo Aditivo y Multiplicativo**: Comparar estos enfoques para determinar cuál describe mejor la evolución de los precios.
  - **Modelo ARIMA**: Utilizar **ARIMA** para modelar la tendencia y la estacionalidad, y así obtener predicciones precisas sobre el precio futuro.
  - **Modelo SARIMAX**: Incluir **variables exógenas**, como el volumen de acciones negociadas, en el modelo para mejorar la precisión de las predicciones.


### 📊 5. Evaluación y Selección del Mejor Modelo

- **Métricas de Rendimiento**:
  - **MAE** (Error Absoluto Medio): Para medir la magnitud promedio del error.
  - **RMSE** (Raíz del Error Cuadrático Medio): Para evaluar la dispersión de los errores.
  - **MAPE** (Error Absoluto Porcentual Medio): Para evaluar el error relativo en la predicción.

### 📚 6. Interpretación del Modelo Seleccionado

- **Análisis de la Predicción**:
  - Utilizar el modelo seleccionado para realizar predicciones del **precio de cierre** y analizar cómo la **tendencia**, **estacionalidad** y **factores externos** afectan los precios.

- **Identificación de Patrones**:
  - Analizar los patrones estacionales y los picos de **volumen** para ayudar a los inversores a identificar **momentos clave** para entrar o salir del mercado.


### 📝 Lograremos

Este modelo predictivo permitirá a los inversores identificar **patrones de crecimiento y contracción** en el precio de las acciones de Tesla, brindándoles una ventaja para:

- **Anticipar Subidas de Precio**: Aprovechar los períodos de alta demanda y crecimiento en los precios, identificados mediante los modelos SARIMAX y ARIMA.
- **Evitar Caídas Significativas**: Detectar patrones de caída en el precio mediante el análisis de la **tendencia** y la **estacionalidad**, ayudando a los inversores a **gestionar el riesgo** y reducir pérdidas potenciales.
- **Momento Óptimo para Comprar o Vender**: Utilizar los picos de **volumen** y la información estacional para determinar cuándo realizar operaciones de compra o venta.

En el presente script, trabajaremos con acciones de Tesla por lo que importaremos la serie directamente de Yahoo Finance: https://finance.yahoo.com/quote/TSLA?p=TSLA&.tsrc=fin-srch, si prefieren importar la serie directamente a nuestro entorno de trabajo, por favor instalen el module: pandas-datareader, via pip para que podamos llamarlo localmente.

In [ ]:
#Llamemos al module
!pip install yfinance

In [ ]:
##!pip install --upgrade yfinance

---

## 🖥️ **Desarrollo de la Serie de Tiempo** 💻

In [ ]:
import yfinance as yf

# Set the ticker symbol
ticker_symbol = 'TSLA'

# Request the data from Yahoo Finance
df = yf.download('TSLA', start='2000-01-01', end='2025-07-31')

# Print the data info
print(df.info())

Perfecto hemos recopilado la data de las cotizaciones diarias de las acciones de Tesla desde el año 2010  hasta el presente año, mes y día, por lo que tenemos un total de 3240 observaciones sin valores perdidos y 5 columnas: su valor a la apertura en la bolsa, el valor máximo de cotización en ese día, el valor mínimo de cotización en ese d8ía, el precio de cierre, el precio ajustado de cierre, que refleja el valor de esa acción después de contabilizar cualquier acción corporativa en ese día, y el volumen de las acciones que se comercializaron.

Como es habitual procedamos a reemplazar los nombres de las columnas a español:

#### Recodificar el nombre de las columnas a español

In [ ]:
#df.rename({'Open':'precio_apertura', 'High':'precio_max', 'Low': 'precio_min',
#           'Close': 'precio_cierre', 'Adj Close':'precio_ajustado_cierre', 'Volume': 'Volumen'}, axis=1, inplace=True)

In [ ]:
df.columns

In [ ]:
# Convertir el MultiIndex a nombres de columnas concatenados con un solo "_"
df.columns = ['_'.join(col).strip().replace(' ', '_').replace('__', '_') for col in df.columns]

# Renombrar correctamente las columnas
df.rename(columns={
    'Open_TSLA': 'precio_apertura',
    'High_TSLA': 'precio_max',
    'Low_TSLA': 'precio_min',
    'Close_TSLA': 'precio_cierre',
    'Volume_TSLA': 'Volumen'
}, inplace=True)

# Mostrar primeras filas para verificar cambios
print(df.head())


In [ ]:
df

#### Importemos las librerías necesarias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

#### Grafiquemos nuestras series de tiempo

In [ ]:
df.plot(subplots=True, figsize=(12,15))
plt.title('Evolución de las acciones de Tesla')

plt.show()

# **Los cuatro componentes de una serie de tiempo**

Dado que el análisis de series de tiempo implica comprender varios aspectos sobre la naturaleza de la serie para predecir valores futuros, a continuación se presentan los cuatro componentes teóricos de las series de tiempo que corresponden a sus características.

## 1. Tendencia o componente tendencial

El componente de tendencia recoge las variaciones en los datos con la variable de tiempo o la frecuencia de los datos, por lo que le permite analizar el aumento o la disminución de los datos a lo largo del tiempo.
Como ejemplos podríamos tener a las cotizaciones en bolsa de las acciones de las empresas, las ventas, entre otras.



En otras palabras, la tendencia es la dirección a largo plazo de la serie temporal y visualmente cuando hay una tendencia marcada se assemeja a la siguiente figura:

![image.png](https://raw.githubusercontent.com/marcecevallos/UDLA_Analitica_predictiva/main/An%C3%A1lisis%20de%20series%20de%20tiempo/tendencia.png)

_Fuente: Australian transport assessment and planning: Forecasting and evaluation_

## 2. Ciclo o componente cíclico

Son oscilaciones o componentes recurrentes en la serie de tiempo que pueden o no ser periódicas. Si las oscilaciones duran más de un año se consideran cíclicas.

Este componente, suelen representar a las etapas de bonanza económica así también como las crisis económicas, sin embargo, los ciclos económicos resultan difícil de reconocer, por lo que en la práctica, muchas veces no se consiguen separar de la tendencia y visualmente se representa así:

![image.png](https://raw.githubusercontent.com/marcecevallos/UDLA_Analitica_predictiva/main/An%C3%A1lisis%20de%20series%20de%20tiempo/ciclo.png)

_Fuente: Australian transport assessment and planning: Forecasting and evaluation_

## 3. Estacionalidad o componente estacional

La estacionalidad es el componente periódico de la serie de tiempo que ocurre dentro de un año, que identifica a las variaciones que ocurren a intervalos regulares de tiempo, como ocurren alrededor del mismo período de tiempo. Como ejemplos tenemos a feriados, temporadas, festivales, entre otros.

Visualmente el componente estacional se ve aprecia como la siguiente figura:


![image.png](https://raw.githubusercontent.com/marcecevallos/UDLA_Analitica_predictiva/main/An%C3%A1lisis%20de%20series%20de%20tiempo/seasonal.png)

_Fuente: Australian transport assessment and planning_

#### Propiedad de estacionariedad

Una serie de tiempo que tiene las mismas propiedades estadísticas a lo largo del tiempo es estacionaria, requisito en el análisis de series de tiempo.

Este es un requisito, en la medida que en su ausencia, un modelo que describa los datos variará en precisión en diferentes momentos, por lo que para que el modelo sea confiable, la media, varianza y correlaciones, deben poder describir con precisión los datos en todos los puntos de interés en el tiempo.

## 4. Componente irregular

El componente irregular corresponde a las variaciones aleatorias en la serie de tiempo. Es conocido también como "ruido" y recoge alteraciones de la serie, que no responden a ninguno de los anteriores componentes, por lo que recoge todo lo que no queda explicado por el modelo. Como ejemplos de este componente podemos tener a circunstancias imprevistas como desastres naturales o pandemias de corta duración.

Gráficamente se visualizan como:

![image.png](https://raw.githubusercontent.com/marcecevallos/UDLA_Analitica_predictiva/main/An%C3%A1lisis%20de%20series%20de%20tiempo/ruido.png)

_Fuente: Australian transport assessment and planning_

En sí el componente irregular, o error es lo que queda de la serie temporal después de eliminar los otros componentes y la estacionariedad.

Por lo que, la serie de los residuos, es una serie temporal también que recoge la diferencia entre un valor observado y un valor predicho, a partir de un modelo de serie temporal, en un momento determinado. El punto clave es que si nuestro modelo de serie de tiempo, es capaz de explicar la correlación serial en las observaciones, entonces los residuos no están correlacionados serialmente.

Esto significa que cada elemento de la serie residual no correlacionada en la serie es una realización independiente de alguna distribución de probabilidad. Es decir, los residuos mismos son independientes e idénticamente distribuidos (i.i.d.). Esto nos lleva al concepto de Random Walk.

### Random Walk o paseo aleatorio

Un paseo aleatorio es un modelo de series de tiempo en el que la observación actual es igual a la observación anterior con un paso aleatorio hacia arriba o hacia abajo, por lo que la serie no se puede predecir sobre la base de la historia pasada. En otras palabras, es un proceso estocástico que consiste en la suma de una secuencia de cambios en una variable aleatoria. Estos cambios no están correlacionados con cambios pasados, lo que significa que no existe un patrón en los cambios en la variable aleatoria y estos cambios no se pueden predecir.

Se define formalmente a continuación:

$$ X_t = X_{t_-1} + w_t $$

Donde 𝑤𝑡 es ruido blanco o una serie de tiempo donde los elementos son independientes e idénticamente distribuidos (i.i.d.), con una media de cero, varianza y sin correlación serial, entonces decimos que la serie temporal es un ruido blanco discreto (DWN). En resumen, el ruido blanco es cualquier distribución que tenga:
- Cero media
- Una varianza/desviación estándar constante (no cambia con el tiempo)
- Autocorrelación cero en todos los retrasos

Por lo que, es una serie de números aleatorios y, por definición, ningún algoritmo puede modelar razonablemente su comportamiento.

_Nota No.2: Una confusión común es pensar en una caminata aleatoria es una simple secuencia de números aleatorios. Este no es el caso porque, en una caminata aleatoria, cada paso depende del paso anterior. Por esta razón, la autocorrelación es distinta de cero._

##  **Descomposición de la serie de tiempo por el modelo de Aditivo**

El valor de la serie temporal suele ser una combinación de los componentes anteriores en cualquier punto del tiempo, y pueden ser producto de la suma de todos los componentes, la multiplicación o una interacción entre las dos operaciones. Por ello, existen tres modelos comunmente usados en el análisis de series de tiempo:

Un modelo aditivo sugiere que los componentes de las serie de tiempo se pueden adicionar ya que  la amplitud de las variaciones estacionales e irregulares no cambia a medida que el nivel de la tendencia sube o baja. En tales casos, un modelo aditivo es apropiado.

Matemáticamente se describe como:

$$ Y(t) = Tendencia + estacionalidad + ciclo + componente.irregular $$

 Si la estacionalidad y los residuos son independientes de la tendencia en la serie de tiempo graficada, usamos un modelo aditivo para descomponer los datos. Gráficamente un modelo aditivo se asemeja a la siguiente figura:

![image.png](https://raw.githubusercontent.com/marcecevallos/UDLA_Analitica_predictiva/main/An%C3%A1lisis%20de%20series%20de%20tiempo/additive%20vs%20multiplicative.png)

_Fuente: Analyticsvidhya: Time Series Forecasting | Various Forecasting Techniques_

_Nota No. 1: La diferencia entre el modelo aditivo y el multiplicativo, radica en su estructura, tal como el aditivo suma todos los componentes, el modelo multiplicativo los multiplica: 𝑌(𝑡)= 𝑇𝑒𝑛𝑑𝑒𝑛𝑐𝑖𝑎 * 𝑐𝑖𝑐𝑙𝑜 * 𝑒𝑠𝑡𝑎𝑐𝑖𝑜𝑛𝑎𝑙𝑖𝑑𝑎𝑑 * 𝑐𝑜𝑚𝑝𝑜𝑛𝑒𝑛𝑡𝑒.𝑖𝑟𝑟𝑒𝑔𝑢𝑙𝑎𝑟_

_Un modelo aditivo es lineal, donde los cambios a lo largo del tiempo se realizan de manera constante en la misma cantidad. Por su parte, en los modelos multiplicativos, los cambios aumentarían o disminuirían de forma no lineal con el tiempo, siendo estos cuadráticos o exponenciales._

### Manejo y descomposición de la serie en Python

#### Visualización de la serie de tiempo

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df.precio_cierre)
plt.title("Evolución de las acciones de Tesla")
plt.xlabel("tiempo")
plt.ylabel("precio al cierre")
plt.show()

Como podemos observar, la evolución de las acciones al cierre de TESLA a partir de 2020 se asemeja más a un modelo aditivo que a uno multiplicativo. Sin embargo, los datos desde su cotización en bolsa hasta antes del año 2020, son relativamente planos.

Apriori a partir del año 2020 notamos que la serie tiene una tendencia creciente y parece tener un componente cíclo dentro de un período menor a un año por lo que podríamos jugar con el argumento de período en nuestro análisis a seguir.

In [ ]:
df['fecha'] = pd.to_datetime(df.index)

In [ ]:
df.fecha[0]

In [ ]:
df.describe()

El análisis estadístico del dataset de las acciones de Tesla proporciona información sobre la distribución de varias columnas financieras como los precios y el volumen. A continuación, se presentan algunas observaciones clave:

**Resumen Estadístico**
- **Número de Datos (count)**: Cada una de las variables contiene un total de **3,592** entradas, lo que indica que los datos cubren **3,592 días** de transacciones.
- **Promedio (mean)**:
  - Los precios de apertura, cierre, máximo, y mínimo tienen valores cercanos entre sí (entre **75.64** y **79.14**), lo que sugiere estabilidad relativa en el precio promedio de la acción durante este período.
  - El volumen promedio es de **9.67 millones**, lo que indica que, en promedio, hubo un alto nivel de negociación diaria.

- **Mínimo (min)** y **Máximo (max)**:
  - El **precio_apertura** más bajo registrado fue de **1.08**, y el precio máximo alcanzado fue de **414.50**. Esta amplia diferencia sugiere un crecimiento significativo en el precio de las acciones de Tesla durante el período cubierto por los datos.
  - El **volumen** mínimo registrado fue **1.78 millones**, mientras que el máximo fue **91.41 millones**. Esto muestra una gran variabilidad en el volumen diario de transacciones, lo cual puede estar relacionado con anuncios importantes, eventos económicos, o noticias relevantes de Tesla.

- **Mediana (50%)**:
  - La **mediana** de los precios de apertura y cierre es de **17.59**, lo cual es mucho menor que el valor máximo, indicando que el precio de las acciones se ha incrementado notablemente en los últimos períodos, mostrando un comportamiento altamente sesgado hacia precios más altos.
  - Para el **volumen**, la mediana es de **82 millones**, lo cual está en línea con la media y confirma una fuerte actividad comercial.

- **Desviación Estándar (std)**:
  - La **desviación estándar** de los precios es bastante alta (entre **100** y **105**), lo que indica una gran variabilidad y volatilidad en el precio de las acciones durante el periodo de tiempo cubierto.
  - La desviación estándar del **volumen** es de **7.82 millones**, lo cual también refleja una significativa volatilidad en la cantidad de acciones negociadas diariamente.

**Cuartiles (25%, 50%, 75%)**
- **Primer Cuartil (25%)**:
  - Los precios del **25% de los días** se encuentran por debajo de los **11.80 - 12.07**, lo cual sugiere que durante un período significativo, las acciones se negociaron a precios bajos.
  
- **Tercer Cuartil (75%)**:
  - Los precios del **75% de los días** se encuentran por debajo de los **171.74 - 175.08**, lo cual muestra que solo en el 25% restante se alcanzaron valores altos, reflejando el crecimiento reciente.

**Observaciones Clave**
1. **Volatilidad Alta**:
   - La **alta desviación estándar** y la diferencia significativa entre los valores máximos y la mediana en todos los precios indican que el precio de las acciones de Tesla es altamente volátil. Esto es común en acciones de empresas tecnológicas innovadoras y de crecimiento rápido.

2. **Tendencia Creciente**:
   - Los valores mínimos y máximos de las variables de precios muestran un **gran incremento** a lo largo del tiempo, lo cual indica que Tesla ha tenido un crecimiento notable en el valor de sus acciones.

3. **Alta Actividad Comercial**:
   - La variabilidad en el volumen y los valores altos del volumen promedio y máximo sugieren que las acciones de Tesla son muy populares entre los inversores, con períodos de **alto interés** probablemente relacionados con noticias o anuncios de la compañía.

**Recomendaciones preliminares para Inversores**
- **Monitorear la Volatilidad**: Dada la alta volatilidad de los precios de Tesla, se recomienda a los inversores **monitorear las noticias y los eventos relevantes** que puedan influir en los precios. Es importante considerar esta volatilidad antes de realizar inversiones a corto plazo.
- **Aprovechar las Tendencias**: Los precios muestran una tendencia creciente significativa. Los inversores a largo plazo podrían aprovechar la **tendencia positiva** en el precio de las acciones.
- **Estar Atentos a los Picos de Volumen**: Los altos volúmenes de negociación suelen coincidir con anuncios importantes, por lo que es recomendable **seguir de cerca** esos picos para identificar oportunidades de inversión o riesgos potenciales.


In [ ]:
df.isna().sum()

Nos aseguramos de que la base de datos no tenga valores nulos.

**Descomposición de las acciones de TESLA**

Como la mayoría de las series de tiempo no son estacionales, y este es un requisito para modelarlas, tenemos que recurrir a técnicas de descomposición para separar los componentes y lograr que los componentes no dependan del tiempo.

Existen varios métodos para lograr el acometido, por ejemplo: remover la tendencia con trasnformaciones a la data, restar rolling means, o realizar el análisis de descomposición de componentes por statsmodels:

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from pylab import rcParams
rcParams['figure.figsize'] = 11, 9

El argumento de period es uno de los más importantes de esta función, y aunque no tenemos forma de conocerlo, podemos intuirlo. Varios artículos sugieren que cuando se tiene datos diarios como en nuestro caso se pruebe con 3 opciones:

- 5  : porque ese es el número de días que la bolsa se encuentra abierta por semana,
- 20 : porque es el número de días que  la bolsa se encuentra abierta por mes, y
- 253: porque es el número de días que  la bolsa se encuentra abierta por año aproximadamente

Para este ejemplo, iniciaremos con la de menor granuladidad para observar qué resultados obtenemos.

In [ ]:
c = df[df['fecha'].dt.year >= 2019]

In [ ]:
c

In [ ]:
plt.plot(c.precio_cierre)
plt.title("Evolución de las acciones de Tesla")
plt.xlabel("tiempo")
plt.ylabel("precio al cierre")
plt.show()

**Análisis del Gráfico de Evolución del Precio de las Acciones de Tesla**

1. **Crecimiento Rápido (2020-2021)**:
   - Tesla experimentó un crecimiento explosivo desde principios de **2020** hasta mediados de **2021**, con el precio alcanzando picos cercanos a **400 dólares**. Este aumento fue impulsado por el interés de los inversores y la adopción de vehículos eléctricos.

2. **Correcciones y Alta Volatilidad (2022-2023)**:
   - Posterior al pico de 2021, el precio experimentó una **corrección significativa** y una alta **volatilidad** en 2022-2023. Esto refleja la incertidumbre del mercado, cambios macroeconómicos y competencia creciente.

3. **Recuperación Parcial (2024)**:
   - En **2024**, se observa una recuperación parcial del precio, aunque sin alcanzar los máximos anteriores de 2021. Esto podría reflejar mejoras operativas y una mayor estabilidad en la producción.

**Conclusiones Estratégicas**
- **Inversores a Largo Plazo**: Tesla muestra potencial de crecimiento a largo plazo. Se recomienda monitorear sus avances tecnológicos.
- **Inversores a Corto Plazo**: La volatilidad presenta oportunidades para el **trading**; es crucial estar atentos a noticias y resultados trimestrales.
- **Riesgo**: La alta **volatilidad** sugiere que se deben considerar estrategias de cobertura para mitigar riesgos.


In [ ]:
result1 = seasonal_decompose(df.precio_cierre[df.index>"2020"], model='additive', period=253)
# La frecuencia es anual
# period=253 indicaría que estás buscando patrones estacionales anuales en los datos.

In [ ]:
result1.plot()
plt.show()

La primera gráfica representa nuestra serie de Tesla original, mientras que el segundo gráfico representa la tendencia de nuestro modelo de serie temporal extraído de la serie original. Finalmente, los gráficos tercero y cuarto representan la estacionalidad y los componentes residuales en la misma serie de tiempo.

Perfecto! Interpretemos los gráficos de resultados:

- Del gráfico de tendencia observamos que efectivamente existe una tendencia creciente
- Del gráfico de ciclo, se puede observar de igual manera que existe un componente estacional
- Del gráfico de los residuos, el componente  parece ser aleatorio

#### Extraigamos los componentes de la serie de tiempo

In [ ]:
tendencia1 = result1.trend
estacional1 = result1.seasonal
residuo1 = result1.resid

In [ ]:
pd.set_option('display.max_rows', None)

#### Rolling centered window

Este comportamiento se da por la ventana deslizante centrada a los datos, así, si se tiene:

10,20,30,40,50  y se aplica la ventana deslizante de tamaño 3, la primera ventana será 10,20,30, y el valor se colocará en el segundo dato y así sucesivamente

#### Importemos la prueba de Dickey Fuller para validar estacionariedad

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
X1 = residuo1.dropna()
result_DF = adfuller(X1)
print('ADF Statistic: %f' % result_DF[0])
print('p-value: %f' % result_DF[1])
print('Critical Values:')
for key, value in result_DF[4].items():
    print('\t%s: %.3f' % (key, value))

H0: La serie tiene una raíz unitaria (no estacionaria)

H1:  La serie no tiene una raíz unitaria (estacionaria).

Ejecutar la función, imprime el valor de la estadística de prueba de -3.43, cuanto más negativa sea esta estadística, más probable es que rechacemos la hipótesis nula (Ho: No tenemos un conjunto de datos estacionario).

Como parte del resultado, obtenemos una tabla de consulta para ayudar a determinar la estadística ADF. Podemos ver que nuestro valor estadístico de -3.44 es menor que el valor de -3.19 al 1%, y a -2.86 al 5%, por lo que podemos rechazar la hipótesis nula con un nivel de significancia del 1% y 5%.

Rechazar la hipótesis nula significa que el proceso tiene raíz unitaria y, a su vez, que la serie temporal es estacionaria o no tiene una estructura dependiente del tiempo.

### Para finalizar:

Comparemos los demás modelos con los otros dos períodos y escojamos el modelo que alcance el menor p-valor en la prueba de Dickey Fuller.

In [ ]:
result2 = seasonal_decompose(df.precio_cierre[df.index>"2020"], model='additive', period=20)

In [ ]:
result2.plot()
plt.show()

Perfecto! Interpretemos los gráficos de resultados:

- Del gráfico de tendencia observamos que efectivamente existe una tendencia creciente aunque menos clara que con el parámetro de 253
- Del gráfico de ciclo, se puede observar de igual manera que existe un componente estacional
- Del gráfico de los residuos, el componente  parece ser aleatorio más en la última sección de la serie

#### Extraigamos los componentes de la serie de tiempo

In [ ]:
tendencia2 = result2.trend
estacional2 = result2.seasonal
residuo2 = result2.resid

#### Importemos la prueba de Dickey Fuller para validar estacionariedad

In [ ]:
X2 = residuo2.dropna()
result_DF2 = adfuller(X2)
print('ADF Statistic: %f' % result_DF2[0])
print('p-value: %f' % result_DF2[1])
print('Critical Values:')
for key, value in result_DF2[4].items():
    print('\t%s: %.3f' % (key, value))

**🔹 Hipótesis del test**
1. **Hipótesis nula (\(H_0\))**:  
   > La serie temporal **tiene una raíz unitaria**, es decir, **no es estacionaria** (tiene tendencia o dependencia en el tiempo).

2. **Hipótesis alternativa (\(H_1\))**:  
   > La serie temporal **es estacionaria** (no tiene raíz unitaria), lo que significa que su media y varianza son constantes en el tiempo.

Ejecutar el ejemplo imprime el valor de la estadística de prueba de -8.09, cuanto más negativa sea esta estadística, más probable es que rechacemos la hipótesis nula (Ho: No tenemos un conjunto de datos estacionario).

Como parte del resultado, obtenemos una tabla de consulta para ayudar a determinar la estadística ADF. Podemos ver que nuestro valor estadístico de -8.25 es menor que el valor de -3.44 al 1%, y a -2.864 al 5%, por lo que podemos rechazar la hipótesis nula con un nivel de significancia del 1% y 5%.

Rechazar la hipótesis nula significa que el proceso no tiene raíz unitaria y, a su vez, que la serie temporal es estacionaria o no tiene una estructura dependiente del tiempo.

Para finalizar, comparemos los otros dos modelos con el de mayor granularidad que corresponde al período de 5 o 5 días a la semana.

In [ ]:
result3 = seasonal_decompose(df.precio_cierre[df.index>"2020"], model='additive', period=5)

In [ ]:
result3.plot()
plt.show()

Perfecto! Interpretemos los gráficos de resultados:

- Del gráfico de tendencia observamos que efectivamente existe una tendencia creciente aunque menos clara que con el parámetro de 253
- Del gráfico de ciclo, se puede observar de igual manera que existe un componente estacional pero como el período es pequeño los resultados se encuentran casi sobrepuestos
- Del gráfico de los residuos, el componente  parece ser aleatorio en la última sección de la serie

#### Extraigamos los componentes de la serie de tiempo

In [ ]:
tendencia3 = result3.trend
estacional3 = result3.seasonal
residuo3 = result3.resid

#### Importemos la prueba de Dickey Fuller para validar estacionariedad

In [ ]:
X3 = residuo3.dropna()
result_DF3 = adfuller(X3)
print('ADF Statistic: %f' % result_DF3[0])
print('p-value: %f' % result_DF3[1])
print('Critical Values:')
for key, value in result_DF3[4].items():
    print('\t%s: %.3f' % (key, value))

Ejecutar el ejemplo imprime el valor de la estadística de prueba de -10.43, cuanto más negativa sea esta estadística, más probable es que rechacemos la hipótesis nula (Ho: No tenemos un conjunto de datos estacionario).

Como parte del resultado, obtenemos una tabla de consulta para ayudar a determinar la estadística ADF. Podemos ver que nuestro valor estadístico de -13.65 es menor que el valor de -3.44 al 1%, y a -2.864 al 5%, por lo que podemos rechazar la hipótesis nula con un nivel de significancia del 1% y 5%.

Rechazar la hipótesis nula significa que el proceso  tiene raíz unitaria y, a su vez, que la serie temporal es estacionaria o no tiene una estructura dependiente del tiempo.

### Conclusión:

Si comparamos los p-valor de los 3 modelos, observamos que todos pasan la prueba de estacionariedad por Dickey Fuller. En este caso, escogeremos el modelo con el menor p-valor sin llegar al tercer modelo, ya que los datos del componente estacional se sobreponen, por lo que personalmente me quedaría con el primer modelo dado que capturó de mejor manera la tendencia y ciclo.:

result1 = seasonal_decompose(df.precio_cierre, model='additive', period=253)

### Próximos pasos:

Excelente!

El siguiente paso es predecir, teniendo en cuenta que aunque la técnica de descomposición se usa principalmente para ayudar a analizar y comprender series temporales históricas, también puede resultar útil cuando se intenta un análisis predictivo. Así, y dado que contamos con cada componente separado, podríamos emplear métodos que se adapten mejor a cada componente.

El método a seguir va a ser, tomar los componentes obtenidos previamente, predecirlos individualmente y mediante el método aditivo los sumaremos para obtener nuestra predicción total:

$$ Y(t) = Tendencia + estacionalidad + ciclo + componente.irregular $$

Componentes obtenidos:

In [ ]:
tendencia1.dropna(inplace=True)
estacional1.dropna(inplace=True)
residuo1.dropna(inplace=True)

### Loop para obtener las predicciones mediante el modelo aditivo

In [ ]:
modelo_aditivo = tendencia1 + estacional1 + residuo1
df_componentes = pd.concat([tendencia1, estacional1, residuo1, modelo_aditivo, df.precio_cierre], axis=1)
df_componentes.columns = ['tendencia', 'estacional', 'residuo', 'modelo_aditivo', 'y_observado']
df_componentes.dropna(inplace=True)
df_componentes.tail(8)

Como podemos observar, los componentes de las acciones de TESLA suman nuestro valor observado. Ahora, para probar nuestro modelo, predeciremos 7 días adelante dentro del mes de abril y compararemos con los y_observados o valores reales.

Ahora, pronosticaremos el componente estacional, usando el valor de t-1. (método ingenuo estacional):

In [ ]:
df_predicciones = df_componentes.iloc[-8:,:]
df_predicciones.estacional = df_componentes.iloc[-8:,:].estacional.shift(1)
df_predicciones = df_predicciones.drop('y_observado', axis=1)
df_predicciones[['tendencia', 'residuo', 'modelo_aditivo']] = np.nan
df_predicciones

Los valores del modelo_aditivo, se pronostican entonces usando simplemente la última tendencia y los últimos valores de tendencia y residuo (método ingenuo). Por supuesto, aquí también podríamos usar una técnica más compleja como un modelo ARMA no estacional pero por el momento lo mantendremos sencillo.

In [ ]:
df_predicciones['tendencia'] = df_componentes.loc[df_predicciones.index[-1]].tendencia
df_predicciones['residuo'] = df_componentes.loc[df_predicciones.index[-1]].residuo
df_predicciones['modelo_aditivo'] = df_predicciones['tendencia'] + df_predicciones['estacional'] + df_predicciones['residuo']
df_predicciones['y_observado'] = df_componentes.iloc[-8:,:].y_observado
df_predicciones.dropna(inplace=True)
df_predicciones

### Comparamos las predicciones

In [ ]:
df_predicciones['diferencia']=df_predicciones.y_observado - df_predicciones.modelo_aditivo
df_predicciones

Perfecto, hemos predicho mediante el método básico.

Como podemos ver, el modelo por descomposición tiene algunas fuertes diferencias en el mes de abril del presente año, por ello ahora aplicaremos técnicas más avanzadas como lo son los modelos autoregresivos y de medias móviles.

### Mejoremos las prediciones!

Creemos una funci'on para que genere valores futuros y qque no sea naive `generate_forecast`:

1. `trend` y `seasonal`: son las componentes de tendencia y estacionalidad de la serie temporal. Estas son las series que se han obtenido mediante la descomposición de la serie original.

2. `residual_mean` y `residual_std`: son la media y la desviación estándar del componente de residuos de la serie original. Estos valores se han calculado previamente y se utilizan para generar los residuos aleatorios en la predicción futura.

3. `periods`: es el número de períodos hacia el futuro para los cuales se desea generar la predicción.

4. `future_trend`: esta línea de código genera la tendencia futura, utilizando la última observación de la tendencia conocida, y luego agregando el cambio promedio de la tendencia en cada período futuro. El cambio promedio se calcula dividiendo la diferencia entre la última y la primera observación de la tendencia por el número total de períodos en la serie.

5. `future_seasonal`: esta línea de código extrae los últimos períodos de la componente de estacionalidad conocida, que se utilizarán como la estacionalidad futura en la predicción.

6. `future_residual`: esta línea de código genera una serie de residuos aleatorios con la misma media y desviación estándar que el componente de residuos conocido.

7. Por último, se devuelve la suma de la tendencia futura, la estacionalidad futura y los residuos futuros para cada período.

In [ ]:
tendencia1.tail(3)

In [ ]:
tendencia1.iloc[-1]

In [ ]:
residual_mean = np.mean(residuo1)
residual_std = np.std(residuo1)

In [ ]:
# Define a function to generate future values based on the decomposed components
def generate_forecast(trend, seasonal, residual_mean, residual_std, periods=1):
    future_trend = trend.iloc[-1] + np.arange(1, periods+1) * ((trend.iloc[-1] - trend.iloc[0]) / (len(trend)-1))
    future_seasonal = seasonal.iloc[-periods:]
    future_residual = np.random.normal(residual_mean, residual_std, periods)
    return future_trend + future_seasonal + future_residual

In [ ]:
# Generate a forecast for the next 10 periods
forecast = generate_forecast(tendencia1, estacional1, residual_mean, residual_std, periods=1)

# Print the forecasted values
print(forecast)

In [ ]:
#setear = set_sedd= 1234  semilla

# **Modelos estacionarios univariados**

### Los datos

En el presente script, continuaremos trabajando con acciones de Tesla por lo que importaremos la serie directamente de Yahoo Finance: https://finance.yahoo.com/quote/TSLA?p=TSLA&.tsrc=fin-srch

In [ ]:
#Llamemos al module
import pandas_datareader as pdr

#### Carguemos los datos

In [ ]:
import yfinance as yf

# Set the ticker symbol
ticker_symbol = 'TSLA'

# Request the data from Yahoo Finance
df = yf.download('TSLA', start='2000-01-01', end='2025-07-31')

# Print the data info
print(df.info())

Perfecto hemos recopilado la data de las cotizaciones de las acciones de Tesla desde el año 2017 hasta el presente año y mes, por lo que tenemos un total de 1257 observaciones sin valores perdidos y tres columnas: el valor máximo de cotización en ese día, el valor mínimo de cotización en ese día, su valor a la apertura en la bolsa, su valor al cierre, el volumen de las acciones que se comercializaron en ese día y el valor ajustado al cierre que modifica el precio de cierre de una acción para reflejar el valor de esa acción después de contabilizar cualquier acción corporativa.

Como es habitual procedamos a reemplazar los nombres de las columnas a español:

#### Recodificar el nombre de las columnas a español

In [ ]:
df.rename({'Open':'precio_apertura', 'High':'precio_max', 'Low': 'precio_min',
           'Close': 'precio_cierre', 'Adj Close':'precio_ajustado_cierre', 'Volume': 'Volumen'}, axis=1, inplace=True)

In [ ]:
df.head()

## Modelación de la serie de TESLA

#### Importemos las librerías necesarias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

### 1. Verificar si la serie es estacionaria

Intuitivamente sabemos que la serie no es estacionaria ya que en la sección anterior logramos estacionariedad al descomponerla en sus componentes, pero a manera de ejemplo en el presente script usaremos la prueba ADF para asegurarnos.

Recordemos que, usamos la función adfuller de la biblioteca statsmodels y extraemos la estadística ADF y el valor p. Si el estadístico ADF es un número negativo grande y el valor p es menor que 0.05, entonces nuestra serie es estacionaria. De lo contrario, debemos aplicar transformaciones.

**🔹 Hipótesis del test**
1. **Hipótesis nula (\(H_0\))**:  
   > La serie temporal **tiene una raíz unitaria**, es decir, **no es estacionaria** (tiene tendencia o dependencia en el tiempo).

2. **Hipótesis alternativa (\(H_1\))**:  
   > La serie temporal **es estacionaria** (no tiene raíz unitaria), lo que significa que su media y varianza son constantes en el tiempo.


In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
#utilizamos la variable original para evaluar estacionariedad
result_DF = adfuller(df.precio_cierre[df.index>"2020"])
print('ADF Statistic: %f' % result_DF[0])
print('p-value: %f' % result_DF[1])
print('Critical Values:')
for key, value in result_DF[4].items():
    print('\t%s: %.3f' % (key, value))

Nuestro estadístico de ADF de -2.30 y nuestro valor p es de 0.17. Aquí, la estadística ADF no es un número negativo grande y el valor "p" es mayor que 0,05. Por lo tanto, nuestra serie de tiempo no es estacionaria, y debemos aplicar transformaciones para hacerla estacionaria.

Para que nuestra serie sea estacionaria, intentaremos estabilizar la tendencia aplicando una diferenciación de primer orden:

$$ d^1_t=Y_{t}-Y_{t-1} $$

PASO 1: sacar la primera diferencia

In [ ]:
precio_cierre_diff = df.precio_cierre[df.index>"2020"].diff(1).dropna()

In [ ]:
precio_cierre_diff.head()

In [ ]:
plt.plot(precio_cierre_diff)

Con una transformación aplicada a nuestra serie, podemos probar la estacionariedad nuevamente usando la prueba ADF:

PASO 2: comprobamos estacionariedad

In [ ]:
result_DF_diff = adfuller(precio_cierre_diff)
print('ADF Statistic: %f' % result_DF_diff[0])
print('p-value: %f' % result_DF_diff[1])
print('Critical Values:')
for key, value in result_DF_diff[4].items():
    print('\t%s: %.3f' % (key, value))

Esto da una estadística ADF de -8.92 y un valor p de  0. Por lo tanto, con una estadística ADF negativa grande y un valor p mucho menor que 0,05, podemos decir que nuestra serie es estacionaria.

Nuestro siguiente paso es trazar la función de autocorrelación y autocorrelación parcial. La biblioteca statsmodels incluye convenientemente la función plot_acf para nosotros. Simplemente pasamos nuestra serie diferenciada y especificamos el número de retrasos en el parámetro de retrasos.

### **Graficar el ACF y el PACF**

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
plt.rcParams["figure.figsize"] = (8,5)

In [ ]:
plot_acf(precio_cierre_diff, lags=20);
plt.tight_layout()

In [ ]:
plot_pacf(precio_cierre_diff, lags=9);
plt.tight_layout()

Como podemos observar no existe un comportamiento claro en los gráficos de ACF y PACF, esto en gran medida, se debe a que estamos trabajando con una serie de tiempo diaria que recoge mucho ruido y variabilidad. Recuerden que en la primera tarea de esta semana se pidió trabajar o con una serie semanal o mensual, por lo que en este script ejemplificaremos que sucede con una serie mensual para disminuir las distorsiones excesivas diarias propias de las acciones e incrementar así, su poder predictivo.

#### AR (p)

Para el proceso autoregresivo y conforme a la tabla de la sección teórica, requerimos que el ACF decaiga hacia cero y que haya una posible oscilación, la cual existe por lo cual sabemos que esta serie de tiempo es autoregresiva.

Para su orden, inspeccionamos el gráfico de PACF, donde los rezagos 7 y 9 muestran cierta significancia, aunque de forma leve. A partir de esos puntos, la mayoría de los rezagos se mantienen dentro del intervalo de confianza, indicando baja correlación parcial.

En el gráfico de ACF, todos los rezagos se encuentran dentro del intervalo de confianza, lo que sugiere que no hay autocorrelación significativa en los residuos.

Esto indica que la serie residual está bien comportada en términos de independencia temporal, y **no es necesario añadir más términos autorregresivos ni de media móvil**.


#### MA (q)

Para el proceso de medias móviles y conforme a la tabla de la sección teórica, requerimos que el PACF decaiga hacia cero y que haya una posible oscilación. Este componente es menos claro que en el proceso autoregresivo ya que existen varios rezagos que rompen los intervalos de confianza.

Como podemos ver en el ACF, los únicos rezagos significativos son el 7 y el 9. Probaremos inicialmente con MA(7)

### **Determinar el modelo**

Teniendo en cuenta que partimos de una serie diferenciada, nuestro modelo contará con el componente de integración 1, dado que alcanzó la estacionariedad aplicado las primeras diferencias. Además, en virtud que encontramos autocorrelaciones y aurocorrelaciones parciales significativas, probaremos un modelo ARIMA importando la función arima y especificando loc componentes "p", "q" y "d" que corresponde a la diferenciación.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

Recordemos que AR está en función de (p) y MA en función de (q):

In [ ]:
#Definimos el orden de los componentes:
p = 7 #  Un valor de p de 7 significa que se utilizarán los últimos 7 valores de la serie temporal para predecir el siguiente valor.
d = 1 # Un valor de d de 1 indica que se realizará una diferenciación de primer orden para hacer que la serie temporal sea estacionaria.
q = 7 #Un valor de q de 7 significa que se utilizarán los últimos 7 errores de predicción para predecir el siguiente valor.

In [ ]:
modelo1 = ARIMA(df.precio_cierre[df.index>"2020"], order=(p, d, q))
resultado1 = modelo1.fit()
print(resultado1.summary())

Como podemos observar, recibimos un mensaje en el que la optimización falla, y producto de ello, todos los coeficientes son no-significativos o la mayorìa de ellos.

He aquí donde tenemos que intentar varias combinaciones. Mi primera sugerencia sería eliminar el proceso de medias móviles ya que es aquel que es menos significativo y ver que nos devuelve el modelo:

In [ ]:
#Definimos el orden de los componentes:
p = 0 #eliminamos el proceso AR
d = 1
q = 0 # eliminando el proceso MA

In [ ]:
modelo1 = ARIMA(df.precio_cierre[df.index>"2020"], order=(p, d, q))
resultado1 = modelo1.fit()
print(resultado1.summary())

#### verificando a priori nuestro guess con: Autoarima!!!

In [ ]:
from pmdarima import auto_arima

In [ ]:
# Create an auto ARIMA model
model = auto_arima(df.precio_cierre[df.index>"2020"], seasonal=False, suppress_warnings=True, error_action="ignore")

# Print the model summary
print(model.summary())


Muchísimo mejor.

El resumen del modelo proporciona varias medidas estadísticas para evaluar el rendimiento de nuestro modelo, por ejemplo: los parámetros de los coeficientes son altamente significativos ya que todos tienen un valor p inferior a 0,05. por lo que, nos quedaremos con este modelo, y pronosticaremos los valores con esta definición matemática.

### **Predicciones modelo AR**

Generar las predicciones en datos que el modelo nunca ha visto separando la base de datos en 70% y 30%

In [ ]:
train_data=df.precio_cierre[0:int(len(df.precio_cierre)*0.7)]
test_data=df.precio_cierre[int(len(df.precio_cierre)*0.7):]

### For loop para generar cada predicción

In [ ]:
historico = train_data.iloc[:, 0].values.tolist() #
predicciones = []
n = len(test_data)
for time_point in range(n):
    model = ARIMA(historico, order=(p,d,q))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predicciones.append(yhat)
    true_test_value = test_data.iloc[time_point, 0]
    historico.append(true_test_value)

### Gráfico comparativo de las predicciones punto a punto

In [ ]:
rango = df.precio_cierre[int(len(df.precio_cierre)*0.7):].index
plt.plot(rango, predicciones, color='blue', marker='o', linestyle='dashed', label='predicciones')
plt.plot(rango, test_data, color='red', label='valores observados')
plt.title('TESLA precios')
plt.xlabel('Fecha')
plt.ylabel('precios')

plt.legend()
plt.show()

El modelo solo diferenciado, parece predecir decentemente a una frecuencia diaria, ya que los valores reales se encuentran fluctuando alrededor de los valores pronosticados.

Grafiquemos los residuos para asegurarnos de que no haya patrones, intentando identificar la media y la varianza constantes:

In [ ]:
residuos = pd.DataFrame(resultado1.resid)
fig, ax = plt.subplots(1,2, figsize=(15,4))
residuos.plot(title = "Residuos", ax = ax[0])
residuos.plot(kind = 'kde', title = 'Densidad', ax = ax[1])
plt.show()

En el ejemplo anterior, hemos trazado los errores residuales y los gráficos de densidad. Podemos observar que los errores residuales parecen oscilar al alrededor de cero aunque no exctamente y una varianza uniforme en los años posteriores.

El análisis de los residuos del modelo actual muestra cierta **variabilidad** y **dispersión**, lo cual sugiere que todavía existen patrones en los datos que no han sido completamente capturados por el modelo. Aunque los residuos parecen estar centrados alrededor de cero, la densidad muestra una distribución con posibles **anomalías** y **valores extremos**, lo cual indica que es necesario explorar un enfoque más robusto. Por ello, es importante evaluar cómo **performa el modelo SARIMAX**, el cual tiene la capacidad de incorporar **factores externos** (variables exógenas) que podrían mejorar la precisión y la capacidad predictiva del modelo, proporcionando un ajuste más fiel a la realidad de las series de tiempo de Tesla.


# **SARIMAX**

# 📊 Modelo SARIMAX: Explicación y Aplicaciones

## ¿Qué es SARIMAX?
**SARIMAX (Seasonal AutoRegressive Integrated Moving Average with eXogenous variables)** es un modelo estadístico utilizado para analizar y **predecir series temporales**. SARIMAX extiende el modelo **ARIMA** al incorporar tanto la **estacionalidad** como **variables exógenas** que pueden influir en el comportamiento de la serie. Es especialmente útil cuando se tiene una serie de datos que muestra **patrones repetitivos** (como ciclos anuales o mensuales) o cuando existen **factores externos** que afectan la variable que se desea predecir.

## Componentes del Modelo SARIMAX
SARIMAX combina varios componentes para capturar las distintas características de una serie temporal:

- **AR (Autoregressive)**: Representa la dependencia entre un valor y los valores anteriores de la serie. Definido por el parámetro **p**.
- **I (Integrated)**: Indica el número de veces que los datos deben ser diferenciados para lograr una serie estacionaria. Definido por el parámetro **d**.
- **MA (Moving Average)**: Modela la dependencia entre un valor y los errores residuales de observaciones anteriores. Definido por el parámetro **q**.
- **S (Seasonal)**: Captura patrones que se repiten a intervalos regulares (como las estaciones del año). Incluye componentes estacionales autoregresivos, integrados y de promedio móvil, definidos por **P, D, Q** y **m** (donde **m** es la periodicidad del patrón estacional).
- **X (eXogenous Variables)**: Se refiere a la inclusión de **variables exógenas** que afectan la serie, lo que permite modelar el impacto de factores externos en la serie de tiempo.

## Notación de SARIMAX
La notación general para un modelo SARIMAX es:

$$
SARIMAX(p, d, q) \times (P, D, Q, m)
$$

Donde:
- **(p, d, q)**: Representan los componentes ARIMA.
- **(P, D, Q, m)**: Representan los componentes estacionales, y **m** es el número de periodos en el ciclo estacional (por ejemplo, **12** para datos mensuales que muestran un ciclo anual).

## Aplicaciones de SARIMAX
**SARIMAX** se utiliza en una variedad de campos para **predicción y análisis de series temporales**:

- **Finanzas**: Predicción de precios de activos financieros, como acciones o criptomonedas.
- **Ventas y Marketing**: Estimación de la demanda de productos que presenta patrones estacionales, como ventas de ropa de invierno.
- **Producción de Energía**: Predicción del consumo de energía eléctrica, que varía según las estaciones del año y otros factores externos.
- **Clima**: Modelar y predecir variables meteorológicas como temperaturas, precipitaciones y vientos.

## Ventajas del Modelo SARIMAX
- **Captura la Estacionalidad**: A diferencia de ARIMA, **SARIMAX** puede modelar patrones estacionales explícitos, lo cual es crucial para series con repeticiones periódicas.
- **Incorpora Variables Exógenas**: Permite incluir factores externos (como indicadores económicos) que puedan afectar la variable de interés, mejorando la capacidad predictiva.
- **Versatilidad**: Puede ser aplicado a una amplia gama de problemas donde se tiene tanto estacionalidad como influencia de factores externos.

## Conclusión
**SARIMAX** es una herramienta poderosa para el análisis de series temporales complejas, que no solo presentan **tendencias y estacionalidades**, sino que también están influenciadas por **variables externas**. Gracias a su capacidad para capturar estos elementos, **SARIMAX** se convierte en una opción ideal para predecir series temporales en escenarios reales, logrando modelos robustos y con alto poder predictivo.


In [ ]:
from pmdarima import auto_arima
import pandas as pd

# Asegurémonos de que tenemos el DataFrame adecuado
# `df` contiene una columna llamada `precio_cierre` y un índice de fechas

# Filtrar los datos desde 2020
df_filtered = df[df.index > "2020"]

# Crear una variable exógena sintética que tenga más impacto
# Vamos a utilizar el volumen de transacciones, pero suavizado para evitar ruido excesivo
exog_variable1 = df_filtered["Volumen"].rolling(window=3, min_periods=1).mean()
exog_variable = exog_variable1.shift()  # 3 meses de retraso


# Crear un modelo auto ARIMA que considere la estacionalidad (SARIMAX)
sarimax_model = auto_arima(df_filtered['precio_cierre'],
                           exog=exog_variable,   # Variable exógena= Volumen
                           seasonal=True,        # Activar la estacionalidad,  Esta opción indica que queremos incluir un componente de estacionalidad en el modelo, lo cual convierte el modelo de ARIMA a SARIMAX.
                           m=12,                 # Estacionalidad de los datos (por ejemplo, 12 si los datos son mensuales)
                           stepwise=True,        # Utilizar búsqueda escalonada para encontrar los mejores parámetros
                           suppress_warnings=True,
                           error_action="ignore",
                           trace=True)           # Mostrar los pasos para ajustar los modelos

# Imprimir el resumen del modelo
print(sarimax_model.summary())

# Predecir los próximos 12 periodos
forecast = sarimax_model.predict(n_periods=12)

# Mostrar las predicciones
print(forecast)


## 📌 Análisis clave del modelo SARIMAX (TESLA)

- **Modelo seleccionado:** ARIMA(2,1,2)(1,0,1)[12] con variable exógena (*volumen suavizado y retrasado*).
- **Todos los coeficientes son significativos** (`p < 0.05`) → el modelo está bien ajustado.
- **Captura estacionalidad mensual** de forma efectiva (componentes SAR y SMA en L=12).
- **No hay autocorrelación en los residuos** (Ljung-Box `p = 0.29`) ✅.
- **Hay heteroscedasticidad y no normalidad en los residuos** ❌ (Jarque-Bera `p = 0.00`, Heteroscedasticidad `p = 0.00`).
- **Conclusión:** Buen modelo para predicción puntual, pero los intervalos de confianza pueden no ser totalmente confiables.


In [ ]:
# Mostrar resultados usando pandas en lugar de ace_tools
forecast_df = pd.DataFrame({"Fecha": pd.date_range(start=df_filtered.index[-1], periods=12, freq='M'), "Predicción": forecast})
print("Predicción SARIMAX:")
print(forecast_df) # displaying the dataframe using print

El resultado del modelo **SARIMAX** indica que se utilizó una configuración de **ARIMA(2,1,2)(1,0,1)[12]**, lo cual significa que el modelo incluye dos componentes autorregresivos y dos de media móvil, junto con un componente estacional autorregresivo y uno de media móvil, todos con una diferenciación de orden 1 y estacionalidad mensual.

El valor de **sigma²** (85.3691) refleja la varianza del error del modelo.

Las pruebas estadísticas muestran un **p-valor** de la prueba de **Ljung-Box (Q)** superior a 0.05 (0.29), lo cual indica que no hay autocorrelación significativa en los residuos.

Sin embargo, la **prueba de Jarque-Bera** presenta un p-valor de 0.00, lo que sugiere que los residuos no siguen una distribución normal.

Además, se evidencia **heteroscedasticidad** en los errores (p-valor de 0.00).

Estos resultados indican que el modelo ajusta bien la estructura temporal y estacional de la serie, aunque la distribución de los residuos aún puede ser optimizada para mejorar la confiabilidad de los intervalos de predicción.



## 🌟 Ventajas del Uso de `auto_arima` para SARIMAX

- **Automatización**: `auto_arima` permite automatizar la selección de los mejores valores para los componentes **p, d, q** y **P, D, Q** del modelo **SARIMAX**.

- **Ahorro de Tiempo**: En lugar de ajustar manualmente múltiples modelos, `auto_arima` realiza la búsqueda automáticamente, probando varias combinaciones para encontrar la que minimiza algún criterio (como el **AIC**).

- **Estacionalidad**: Incluyendo la estacionalidad (`seasonal=True` y definiendo `m`), `auto_arima` se convierte en una forma efectiva de ajustar modelos **SARIMAX** sin necesidad de especificar manualmente los parámetros estacionales.


In [ ]:
# Crear un DataFrame para almacenar las fechas futuras
# Obtener los valores ajustados del modelo
fitted_values = sarimax_model.predict_in_sample()
future_dates = pd.date_range(df_filtered.index[-1] + pd.DateOffset(1), periods=12, freq='B')
forecast_df = pd.DataFrame({'Fecha': future_dates, 'Predicción': forecast}).set_index('Fecha')

# Graficar los valores reales, los ajustados y las predicciones
plt.figure(figsize=(12, 6))
plt.plot(df_filtered.index, df_filtered['precio_cierre'], label='Valores reales', color='blue')
plt.plot(df_filtered.index, fitted_values, label='Valores ajustados (SARIMAX)', color='yellow', linestyle='--')
plt.plot(forecast_df.index, forecast_df['Predicción'], label='Predicciones', color='red', linestyle='--')
plt.xlabel('Fecha')
plt.ylabel('Precio de Cierre')
plt.title('Predicción del Precio de las Acciones de Tesla con SARIMAX')
plt.legend()
plt.grid(True)
plt.show()




**Análisis del Gráfico de Predicción del Precio de las Acciones de Apple con SARIMAX**

El gráfico muestra la evolución del **precio de cierre** de las acciones de Apple junto con las predicciones del modelo **SARIMAX**. Se presentan tres elementos clave:

1. **Valores Reales (Línea Azul)**: Representan el precio de cierre real de las acciones de Apple desde **2020** hasta **2025**. La línea azul muestra el comportamiento histórico del precio, incluyendo el crecimiento significativo en **2021**, seguido de varias fluctuaciones y un comportamiento volátil entre **2022** y **2023**.

2. **Valores Ajustados (Línea Amarilla)**: Los valores ajustados generados por el modelo SARIMAX se muestran en una línea amarilla discontinua, la cual sigue de cerca los valores reales, lo que indica que el modelo ha logrado capturar la **tendencia general** y la **estacionalidad** de las acciones de Apple. Sin embargo, se observan ciertas desviaciones, particularmente en períodos de alta volatilidad, lo cual indica que el modelo tiene dificultades para capturar movimientos abruptos.

3. **Predicciones (Línea Roja)**: La línea roja representa las **predicciones a futuro** realizadas por el modelo. Esta línea se mantiene bastante alineada con la tendencia de los últimos meses de datos, sugiriendo una expectativa de estabilidad o crecimiento moderado en el futuro cercano. La precisión de estas predicciones dependerá de la estabilidad de los factores exógenos que influyen en el comportamiento de las acciones de Apple.

En resumen, el modelo SARIMAX parece tener un buen rendimiento al capturar la dinámica general del precio de las acciones, pero presenta limitaciones para predecir cambios bruscos. Aún así, es útil para identificar **tendencias a largo plazo** y hacer **proyecciones razonables** que apoyen la toma de decisiones de los inversores.


In [ ]:
import matplotlib.pyplot as plt

# Obtener los residuos del modelo ajustado
residuals = pd.Series(sarimax_model.resid(), index=df_filtered.index[-len(sarimax_model.resid()):])

# Graficar los residuos
plt.figure(figsize=(10, 6))
plt.plot(residuals, color='purple', label='Residuos')
plt.axhline(y=0, color='red', linestyle='--', linewidth=1.5)

# Personalización del gráfico
plt.xlabel('Fecha')
plt.ylabel('Residuos')
plt.title('Gráfico de Residuos del Modelo SARIMAX (auto_arima)')
plt.legend()
plt.grid(True)

# Mostrar el gráfico
plt.show()


**Análisis del Gráfico de Residuos del Modelo SARIMAX (auto_arima)**

El gráfico muestra los **residuos** del modelo SARIMAX a lo largo del tiempo. Los residuos representan la diferencia entre los valores reales y los valores ajustados por el modelo, proporcionando una visión de qué tan bien el modelo ha logrado capturar los patrones de la serie temporal.

1. **Residuos Cercanos a Cero**: La mayor parte de los residuos se encuentran **alrededor de la línea cero** (línea roja), lo cual sugiere que el modelo ha logrado un ajuste razonable, ya que la mayoría de los errores no tienen un sesgo sistemático.

2. **Volatilidad Variable**: Se observa un comportamiento con cierta **volatilidad** en los residuos, especialmente en **2021** y **2022**, donde se registran picos altos, tanto positivos como negativos. Esto indica que hubo periodos donde el modelo no capturó correctamente algunos cambios abruptos, posiblemente debido a eventos externos o movimientos especulativos del mercado.

3. **Distribución Heterogénea**: La presencia de **picos pronunciados** tanto hacia arriba como hacia abajo muestra que, aunque el modelo funciona bien en promedio, hay ciertos períodos donde **predice con errores significativos**.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Obtener los residuos del modelo ajustado
residuals = pd.Series(sarimax_model.resid(), index=df_filtered.index[-len(sarimax_model.resid()):])

# Crear una figura para la densidad
plt.figure(figsize=(10, 6))

# Graficar la densidad de los residuos
sns.kdeplot(residuals, color='purple', shade=True, label='Densidad de los Residuos')

# Personalización del gráfico
plt.xlabel('Residuos')
plt.ylabel('Densidad')
plt.title('Densidad de los Residuos del Modelo SARIMAX (auto_arima)')
plt.axvline(x=0, color='red', linestyle='--', linewidth=1.5)
plt.legend()
plt.grid(True)

# Mostrar el gráfico
plt.show()


La gráfica de densidad de los residuos muestra una clara asimetría hacia la derecha, lo cual indica que el modelo SARIMAX tiene problemas de ajuste, especialmente para algunas observaciones, subestimando los valores reales. Esto sugiere que el modelo no está capturando bien ciertos patrones presentes en los datos o que existen valores atípicos que no se han modelado adecuadamente.

Para mejorar el ajuste, sería recomendable ajustar manualmente los parámetros del modelo o aplicar una transformación a los datos para reducir la asimetría. También se podría revisar la inclusión de posibles variables exógenas o refinar la captura de la estacionalidad para lograr una mejor precisión y obtener residuos más cercanos a una distribución normal centrada en cero

### **Estrategias para Inversionistas**

1. **Aprovechar la Tendencia General**:
   - Los resultados del modelo SARIMAX muestran que las **predicciones del precio** de las acciones de Apple (o Tesla) siguen de cerca la **tendencia general** de los valores reales. Los inversionistas a **largo plazo** podrían beneficiarse de esta capacidad predictiva para identificar **tendencias alcistas** y mantener sus posiciones hasta que el modelo indique una reversión de la tendencia.

2. **Mitigar el Impacto de la Volatilidad**:
   - La presencia de **residuos con picos** que no se ajustan al modelo indica **momentos de alta volatilidad**. Los inversionistas podrían implementar estrategias de **cobertura** o **stop-loss** para proteger sus posiciones durante estos períodos. Identificar estos picos también puede ayudar a evitar momentos de alta incertidumbre, optimizando la entrada y salida del mercado.

3. **Identificación de Oportunidades de Trading a Corto Plazo**:
   - Dado que el modelo ha demostrado cierta **dificultad para capturar cambios abruptos**, los traders a **corto plazo** pueden aprovechar la oportunidad de capitalizar en estos movimientos rápidos, utilizando análisis técnico y aprovechando las **fluctuaciones diarias** que el modelo no predice adecuadamente.

4. **Revisión de Factores Exógenos**:
   - La inclusión de **factores externos** (volumen de transacciones, noticias de la empresa, indicadores macroeconómicos) podría mejorar la precisión del modelo. Los inversionistas pueden utilizar estos factores para **anticipar movimientos de mercado**, y complementar las predicciones del modelo SARIMAX con información relevante que explique los picos en los residuos.

5. **Optimización de Estrategias de Compra y Venta**:
   - Las predicciones realizadas por el modelo SARIMAX ofrecen una guía de **precios proyectados**, que puede ser útil para definir **niveles de entrada y salida** del mercado. Los inversionistas pueden aprovechar los precios predichos para fijar precios objetivo de venta o para planificar compras a precios bajos durante momentos de corrección.

6. **Planificación Basada en Estacionalidad**:
   - Dado que el modelo SARIMAX tiene en cuenta la **estacionalidad**, los inversionistas pueden planificar sus estrategias considerando posibles **fluctuaciones estacionales** en el precio de las acciones. Esto les permite ajustar sus carteras de inversión según las variaciones esperadas a lo largo del año.